In [1]:
# Dependencies
import os
import numpy as np
import requests
import json
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import datetime as dt
import warnings
import random
warnings.filterwarnings('ignore')

In [2]:
#load 2019 prideroute zone file
rdata = pd.read_csv('prideroute.csv')
routedata=pd.DataFrame(rdata)
routedata

,LocationID,Borough,Zone,service_zone,prideroute
0,1,EWR,Newark Airport,EWR,NaN
1,2,Queens,Jamaica Bay,Boro Zone,NaN
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,NaN
3,4,Manhattan,Alphabet City,Yellow Zone,Y
4,5,Staten Island,Arden Heights,Boro Zone,NaN
...,...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone,NaN
261,262,Manhattan,Yorkville East,Yellow Zone,NaN
262,263,Manhattan,Yorkville West,Yellow Zone,NaN
263,264,Unknown,NV,NaN,NaN


In [3]:
#load 2019 june yellowcab trip data
data = pd.read_csv('yellow_tripdata_2019-06.csv')
tripdata=pd.DataFrame(data)

In [4]:
tripdata.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [5]:
tdata=tripdata[(pd.to_datetime(tripdata["tpep_pickup_datetime"])>"2019-06-22")&(pd.to_datetime(tripdata["tpep_pickup_datetime"])<"2019-07-01")]
tdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
599520,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,13.0,1.0,0.5,3.46,0.00,0.3,20.76,2.5
599521,2,2019-06-24 10:58:03,2019-06-24 11:08:39,1,1.63,1,N,230,142,1,8.5,1.0,0.5,1.50,0.00,0.3,14.30,2.5
599522,2,2019-06-24 11:14:35,2019-06-24 11:18:08,1,0.80,1,N,239,238,2,4.5,1.0,0.5,0.00,0.00,0.3,8.80,2.5
692228,2,2019-06-24 22:25:00,2019-06-24 22:40:32,6,8.53,1,N,237,138,1,25.0,0.5,0.5,6.98,6.12,0.3,41.90,2.5
694129,2,2019-06-24 22:59:13,2019-06-25 06:22:59,6,19.07,2,N,141,132,1,52.0,0.0,0.5,12.28,6.12,0.3,73.70,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6941019,1,2019-06-30 23:23:03,2019-06-30 23:39:48,1,0.90,1,N,68,158,1,11.0,3.0,0.5,2.00,0.00,0.3,16.80,2.5
6941020,1,2019-06-30 23:50:22,2019-06-30 23:57:01,1,0.50,1,N,246,90,2,6.0,3.0,0.5,0.00,0.00,0.3,9.80,2.5
6941021,1,2019-06-30 23:58:32,2019-07-01 00:00:42,1,0.20,1,N,90,186,1,3.5,3.0,0.5,1.45,0.00,0.3,8.75,2.5
6941022,2,2019-06-30 23:23:10,2019-06-30 23:30:45,1,1.38,1,N,140,163,1,7.5,0.5,0.5,2.26,0.00,0.3,13.56,2.5


In [6]:
filterdata=tdata.merge(routedata,left_on="PULocationID",right_on="LocationID")
#filter pickup locations in the range of parade route
filterdata.dropna(inplace=True)

In [7]:
# URL for GET requests to retrieve taxi zone matching data
with open('taxi_zones.json') as f:
    data = json.load(f)
matchdata=json.dumps(data)

In [8]:
data.keys()

dict_keys(['meta', 'data'])

In [9]:
coordata=data["data"][0][10].split(",")[1:-1]
coordata
lat_list=[]
lon_list=[]
for i in coordata:
    lat_text,lon_text=i.split()
    lat_list.append(float(lat_text))
    lon_list.append(float(lon_text))
avg_lat=sum(lat_list)/len(lat_list)
avg_lon=sum(lon_list)/len(lon_list)

In [10]:
zonedata={}
for zone in data["data"]:
#     limit=len(zone[10].split(","))
#     choice=random.randint(2,limit-2)    
#     long,lati=zone[10].split(",")[choice].split()
#     lon=float(long)
#     lat=float(lati)
    coordata=zone[10].split(",")[1:-1]
    lat_list=[]
    lon_list=[]
    for i in coordata:
        lat_text,lon_text=i.split()
        lat_list.append(float(lat_text.replace("((","")))
        lon_list.append(float(lon_text.replace("))","")))
    lat=sum(lat_list)/len(lat_list)
    lon=sum(lon_list)/len(lon_list)
    locid=int(zone[8])
    zonename=zone[12]
    zonedata[locid]=(locid,zonename,lon,lat)
zonedata

{1: (1, 'Newark Airport', 40.690201973912934, -74.1741817304347),
 2: (2, 'Jamaica Bay', 40.61216378184014, -73.81762424077051),
 3: (3, 'Allerton/Pelham Gardens', 40.86417558133605, -73.84648606037803),
 4: (4, 'Alphabet City', 40.72380737920437, -73.97528897228976),
 5: (5, 'Arden Heights', 40.556607934791565, -74.18998814538088),
 6: (6, 'Arrochar/Fort Wadsworth', 40.601109625990055, -74.07180547285517),
 7: (7, 'Astoria', 40.76136529894991, -73.91953425290545),
 8: (8, 'Astoria Park', 40.778232625414546, -73.92339874044116),
 9: (9, 'Auburndale', 40.75018763597315, -73.79024358667372),
 24: (24, 'Bloomingdale', 40.803115923680885, -73.96806728115648),
 10: (10, 'Baisley Park', 40.67777509794184, -73.7904084173483),
 11: (11, 'Bath Beach', 40.60268561333668, -74.00916940577049),
 12: (12, 'Battery Park', 40.70205428849489, -74.01547496482763),
 13: (13, 'Battery Park City', 40.70934000227755, -74.01736563996842),
 18: (18, 'Bedford Park', 40.86891632973801, -73.88945423604756),
 25:

In [11]:
zonedf=pd.DataFrame(zonedata).T
zonedf

,0,1,2,3
1,1,Newark Airport,40.6902,-74.1742
2,2,Jamaica Bay,40.6122,-73.8176
3,3,Allerton/Pelham Gardens,40.8642,-73.8465
4,4,Alphabet City,40.7238,-73.9753
5,5,Arden Heights,40.5566,-74.19
...,...,...,...,...
256,256,Williamsburg (South Side),40.7107,-73.963
259,259,Woodlawn/Wakefield,40.9,-73.8537
260,260,Woodside,40.7463,-73.9059
261,261,World Trade Center,40.7075,-74.014


In [12]:
zonedf.rename(columns={0:"locid",1:"zone",2:"lat",3:"lon"},inplace=True)
zonedf

,locid,zone,lat,lon
1,1,Newark Airport,40.6902,-74.1742
2,2,Jamaica Bay,40.6122,-73.8176
3,3,Allerton/Pelham Gardens,40.8642,-73.8465
4,4,Alphabet City,40.7238,-73.9753
5,5,Arden Heights,40.5566,-74.19
...,...,...,...,...
256,256,Williamsburg (South Side),40.7107,-73.963
259,259,Woodlawn/Wakefield,40.9,-73.8537
260,260,Woodside,40.7463,-73.9059
261,261,World Trade Center,40.7075,-74.014


In [13]:
pkjoindata=filterdata.merge(zonedf,left_on="PULocationID",right_on="locid")
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,LocationID,Borough,Zone,service_zone,prideroute,locid,zone,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.0,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635853,2,2019-06-30 23:04:47,2019-07-01 22:24:37,2,4.85,1,N,24,243,2,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635854,2,2019-06-30 23:03:57,2019-06-30 23:19:08,1,3.04,1,N,24,236,2,...,2.5,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635855,1,2019-06-30 23:03:26,2019-06-30 23:03:47,1,0.00,5,N,24,24,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635856,2,2019-06-30 23:06:41,2019-06-30 23:12:37,1,1.03,1,N,24,41,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681


In [14]:
pkjoindata.rename(columns={"zone":"pkzone","lat":"pklat","lon":"pklon"},inplace=True)
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,LocationID,Borough,Zone,service_zone,prideroute,locid,pkzone,pklat,pklon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.0,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635853,2,2019-06-30 23:04:47,2019-07-01 22:24:37,2,4.85,1,N,24,243,2,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635854,2,2019-06-30 23:03:57,2019-06-30 23:19:08,1,3.04,1,N,24,236,2,...,2.5,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635855,1,2019-06-30 23:03:26,2019-06-30 23:03:47,1,0.00,5,N,24,24,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681
635856,2,2019-06-30 23:06:41,2019-06-30 23:12:37,1,1.03,1,N,24,41,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,40.8031,-73.9681


In [15]:
dojoindata=pkjoindata.merge(zonedf,left_on="DOLocationID",right_on="locid")
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,zone,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633777,2,2019-06-30 22:52:36,2019-06-30 23:38:32,1,15.29,1,N,90,184,1,...,Yellow Zone,Y,90,Flatiron,40.7425,-73.9968,184,Pelham Bay Park,40.8622,-73.7965
633778,2,2019-06-28 22:46:47,2019-06-28 23:43:13,2,20.58,1,N,186,176,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.7489,-73.9921,176,Oakwood,40.5645,-74.1214
633779,1,2019-06-22 03:33:40,2019-06-22 04:20:36,1,18.20,1,N,246,176,1,...,Yellow Zone,Y,246,West Chelsea/Hudson Yards,40.75,-74.0081,176,Oakwood,40.5645,-74.1214
633780,2,2019-06-26 21:44:47,2019-06-26 22:38:42,2,17.57,1,N,87,176,1,...,Yellow Zone,Y,87,Financial District North,40.7046,-74.0056,176,Oakwood,40.5645,-74.1214


In [16]:
dojoindata.rename(columns={"zone":"dozone","lat":"dolat","lon":"dolon"},inplace=True)
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,40.7188,-74.0099,161,Midtown Center,40.7579,-73.9775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633777,2,2019-06-30 22:52:36,2019-06-30 23:38:32,1,15.29,1,N,90,184,1,...,Yellow Zone,Y,90,Flatiron,40.7425,-73.9968,184,Pelham Bay Park,40.8622,-73.7965
633778,2,2019-06-28 22:46:47,2019-06-28 23:43:13,2,20.58,1,N,186,176,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.7489,-73.9921,176,Oakwood,40.5645,-74.1214
633779,1,2019-06-22 03:33:40,2019-06-22 04:20:36,1,18.20,1,N,246,176,1,...,Yellow Zone,Y,246,West Chelsea/Hudson Yards,40.75,-74.0081,176,Oakwood,40.5645,-74.1214
633780,2,2019-06-26 21:44:47,2019-06-26 22:38:42,2,17.57,1,N,87,176,1,...,Yellow Zone,Y,87,Financial District North,40.7046,-74.0056,176,Oakwood,40.5645,-74.1214


In [17]:
pridedata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-30 16:00:00")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-06-30 18:00:00")]

In [18]:
pridedata.sort_values(by=["tpep_pickup_datetime"],inplace=True)
pridedata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
129970,2,2019-06-30 16:00:04,2019-06-30 16:05:15,6,0.80,1,N,137,162,2,...,Yellow Zone,Y,137,Kips Bay,40.7379,-73.9738,162,Midtown East,40.7566,-73.9727
49673,2,2019-06-30 16:00:04,2019-06-30 16:10:36,2,1.37,1,N,186,50,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.7489,-73.9921,50,Clinton West,40.7678,-73.9975
391614,1,2019-06-30 16:00:04,2019-06-30 16:14:11,3,2.20,1,N,113,13,2,...,Yellow Zone,Y,113,Greenwich Village North,40.7322,-73.9944,13,Battery Park City,40.7093,-74.0174
101990,2,2019-06-30 16:00:08,2019-06-30 16:08:36,1,0.57,1,N,107,113,2,...,Yellow Zone,Y,107,Gramercy,40.7369,-73.9843,113,Greenwich Village North,40.7322,-73.9944
581571,2,2019-06-30 16:00:11,2019-06-30 16:05:30,3,1.42,1,N,87,261,1,...,Yellow Zone,Y,87,Financial District North,40.7046,-74.0056,261,World Trade Center,40.7075,-74.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282799,1,2019-06-30 17:59:47,2019-06-30 18:35:14,2,7.30,1,N,79,41,1,...,Yellow Zone,Y,79,East Village,40.7269,-73.9866,41,Central Harlem,40.8032,-73.9507
300004,1,2019-06-30 17:59:51,2019-06-30 18:11:34,2,1.10,1,N,114,125,2,...,Yellow Zone,Y,114,Greenwich Village South,40.7278,-73.9972,125,Hudson Sq,40.7257,-74.0091
491635,2,2019-06-30 17:59:53,2019-06-30 18:17:38,1,2.52,1,N,100,239,2,...,Yellow Zone,Y,100,Garment District,40.7536,-73.9888,239,Upper West Side South,40.7847,-73.9833
71889,1,2019-06-30 17:59:53,2019-06-30 18:12:34,2,1.20,1,N,4,211,2,...,Yellow Zone,Y,4,Alphabet City,40.7238,-73.9753,211,SoHo,40.7237,-74.0022


In [19]:
priorwkdata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-23 16:00:00")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-06-23 18:00:00")]
priorwkdata.sort_values(by=["tpep_pickup_datetime"],inplace=True)
priorwkdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
37711,1,2019-06-23 16:00:02,2019-06-23 16:11:01,1,1.40,1,N,137,107,1,...,Yellow Zone,Y,137,Kips Bay,40.7379,-73.9738,107,Gramercy,40.7369,-73.9843
71605,1,2019-06-23 16:00:02,2019-06-23 16:10:51,2,1.50,1,N,87,211,1,...,Yellow Zone,Y,87,Financial District North,40.7046,-74.0056,211,SoHo,40.7237,-74.0022
505293,1,2019-06-23 16:00:03,2019-06-23 16:02:14,1,0.40,1,N,186,164,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.7489,-73.9921,164,Midtown South,40.7488,-73.9848
140024,1,2019-06-23 16:00:04,2019-06-23 16:03:47,1,0.70,1,N,234,186,2,...,Yellow Zone,Y,234,Union Sq,40.7395,-73.9901,186,Penn Station/Madison Sq West,40.7489,-73.9921
208766,2,2019-06-23 16:00:04,2019-06-23 16:09:40,1,1.83,1,N,125,234,2,...,Yellow Zone,Y,125,Hudson Sq,40.7257,-74.0091,234,Union Sq,40.7395,-73.9901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64419,1,2019-06-23 17:59:54,2019-06-23 18:00:05,1,16.10,5,N,211,211,1,...,Yellow Zone,Y,211,SoHo,40.7237,-74.0022,211,SoHo,40.7237,-74.0022
221475,2,2019-06-23 17:59:55,2019-06-23 18:15:20,2,2.00,1,N,158,148,2,...,Yellow Zone,Y,158,Meatpacking/West Village West,40.7368,-74.0105,148,Lower East Side,40.7182,-73.9914
604607,1,2019-06-23 17:59:56,2019-06-23 18:15:22,1,2.20,1,N,158,100,1,...,Yellow Zone,Y,158,Meatpacking/West Village West,40.7368,-74.0105,100,Garment District,40.7536,-73.9888
106183,2,2019-06-23 17:59:56,2019-06-23 18:07:28,2,1.33,1,N,186,113,2,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.7489,-73.9921,113,Greenwich Village North,40.7322,-73.9944


In [20]:
# Create Engine
engine = create_engine("sqlite:///trip.db", pool_pre_ping=True)

In [21]:
pridedata.to_sql('pridedata', con=engine, if_exists='replace')

In [22]:
test=pd.read_sql("SELECT * FROM pridedata",engine)

In [23]:
test

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,129970,2,2019-06-30 16:00:04,2019-06-30 16:05:15,6,0.80,1,N,137,162,...,Yellow Zone,Y,137,Kips Bay,40.737860,-73.973838,162,Midtown East,40.756573,-73.972685
1,49673,2,2019-06-30 16:00:04,2019-06-30 16:10:36,2,1.37,1,N,186,50,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.748872,-73.992093,50,Clinton West,40.767768,-73.997484
2,391614,1,2019-06-30 16:00:04,2019-06-30 16:14:11,3,2.20,1,N,113,13,...,Yellow Zone,Y,113,Greenwich Village North,40.732156,-73.994365,13,Battery Park City,40.709340,-74.017366
3,101990,2,2019-06-30 16:00:08,2019-06-30 16:08:36,1,0.57,1,N,107,113,...,Yellow Zone,Y,107,Gramercy,40.736902,-73.984339,113,Greenwich Village North,40.732156,-73.994365
4,581571,2,2019-06-30 16:00:11,2019-06-30 16:05:30,3,1.42,1,N,87,261,...,Yellow Zone,Y,87,Financial District North,40.704606,-74.005571,261,World Trade Center,40.707527,-74.014002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3927,282799,1,2019-06-30 17:59:47,2019-06-30 18:35:14,2,7.30,1,N,79,41,...,Yellow Zone,Y,79,East Village,40.726942,-73.986582,41,Central Harlem,40.803170,-73.950658
3928,300004,1,2019-06-30 17:59:51,2019-06-30 18:11:34,2,1.10,1,N,114,125,...,Yellow Zone,Y,114,Greenwich Village South,40.727806,-73.997183,125,Hudson Sq,40.725680,-74.009110
3929,491635,2,2019-06-30 17:59:53,2019-06-30 18:17:38,1,2.52,1,N,100,239,...,Yellow Zone,Y,100,Garment District,40.753642,-73.988840,239,Upper West Side South,40.784729,-73.983344
3930,71889,1,2019-06-30 17:59:53,2019-06-30 18:12:34,2,1.20,1,N,4,211,...,Yellow Zone,Y,4,Alphabet City,40.723807,-73.975289,211,SoHo,40.723711,-74.002227


In [24]:
priorwkdata.to_sql('priorwkdata', con=engine, if_exists='replace')

In [25]:
test2=pd.read_sql("SELECT * FROM priorwkdata",engine)

In [26]:
test2

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,37711,1,2019-06-23 16:00:02,2019-06-23 16:11:01,1,1.40,1,N,137,107,...,Yellow Zone,Y,137,Kips Bay,40.737860,-73.973838,107,Gramercy,40.736902,-73.984339
1,71605,1,2019-06-23 16:00:02,2019-06-23 16:10:51,2,1.50,1,N,87,211,...,Yellow Zone,Y,87,Financial District North,40.704606,-74.005571,211,SoHo,40.723711,-74.002227
2,505293,1,2019-06-23 16:00:03,2019-06-23 16:02:14,1,0.40,1,N,186,164,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.748872,-73.992093,164,Midtown South,40.748767,-73.984805
3,140024,1,2019-06-23 16:00:04,2019-06-23 16:03:47,1,0.70,1,N,234,186,...,Yellow Zone,Y,234,Union Sq,40.739497,-73.990138,186,Penn Station/Madison Sq West,40.748872,-73.992093
4,208766,2,2019-06-23 16:00:04,2019-06-23 16:09:40,1,1.83,1,N,125,234,...,Yellow Zone,Y,125,Hudson Sq,40.725680,-74.009110,234,Union Sq,40.739497,-73.990138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,64419,1,2019-06-23 17:59:54,2019-06-23 18:00:05,1,16.10,5,N,211,211,...,Yellow Zone,Y,211,SoHo,40.723711,-74.002227,211,SoHo,40.723711,-74.002227
7298,221475,2,2019-06-23 17:59:55,2019-06-23 18:15:20,2,2.00,1,N,158,148,...,Yellow Zone,Y,158,Meatpacking/West Village West,40.736836,-74.010518,148,Lower East Side,40.718209,-73.991417
7299,604607,1,2019-06-23 17:59:56,2019-06-23 18:15:22,1,2.20,1,N,158,100,...,Yellow Zone,Y,158,Meatpacking/West Village West,40.736836,-74.010518,100,Garment District,40.753642,-73.988840
7300,106183,2,2019-06-23 17:59:56,2019-06-23 18:07:28,2,1.33,1,N,186,113,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,40.748872,-73.992093,113,Greenwich Village North,40.732156,-73.994365
